In [ ]:
import pandas as pd
import gensim.downloader as api
import numpy as np
from sklearn.cluster import KMeans
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import os

df = pd.read_excel('caracteristicas.xlsx')

Analisis de la variable clase

In [ ]:
descripcion = df['clase'].describe()

frecuencia_valores = df['clase'].value_counts()

# Imprimir la descripcion general de 'clase'
print("Descripcion general de la columna 'clase':")
print(descripcion)


# Imprimir la frecuencia de cada valor unico en 'clase'
print("Frecuencia de cada valor unico en la columna 'clase':")
print(frecuencia_valores)

In [ ]:
unicos = df['clase'].unique().tolist()
len(unicos)

Word Embedding

In [ ]:
# Modelo preentrenado de Word2Vec
model = api.load("word2vec-google-news-300")

# Convertir las palabras en embeddings
word_vectors = []
valid_words = []
for word in unicos:
    if word in model:
        word_vectors.append(model[word])
        valid_words.append(word)
word_vectors = np.array(word_vectors)

len(valid_words)

In [ ]:
# Contar las veces que se repite cada palabra en la columna clase
cont = defaultdict(int)
for palabra in df['clase']:
    cont[palabra] += 1

print("Conteo de las veces que se repite cada palabra en la columna 'clase':")
for p, contar in cont.items():
    print(f"{p}: {contar}")

Agrupación semántica de etiquetas con K-Means

In [ ]:
# Definir el numero de clusteres (se probo con 5, 10, 20)
num_clusters = 10

# Aplicar K-Means a los vectores de las palabras
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(word_vectors)

# Agrupar palabras por sus etiquetas de cluster
clusters = defaultdict(list)
for word, cluster_id in zip(valid_words, kmeans.labels_):
    clusters[cluster_id].append(word)

# Preparar los datos para escribir en el archivo Excel
data = []
for cluster_id, words in clusters.items():
    for word in words:
        data.append({'Cluster': cluster_id, 'Word': word, 'Frequency': cont[word]})

df = pd.DataFrame(data)
df.to_excel('clusters_10.xlsx', index=False)

print(f'Clusters y palabras guardados en clusters_10.xlsx')

In [ ]:
# Reducir la dimensionalidad de los embeddings a 2D usando t-SNE
tsne = TSNE(n_components=2, random_state=0)
word_vectors_2d = tsne.fit_transform(word_vectors)

# Identificar palabras representativas de cada cluster (cercanas al centroide)
representative_words = {}
for cluster_id in range(num_clusters):
    cluster_indices = np.where(kmeans.labels_ == cluster_id)[0]
    cluster_vectors = word_vectors[cluster_indices]
    centroid = kmeans.cluster_centers_[cluster_id]
    closest_word_index = np.argmin(np.linalg.norm(cluster_vectors - centroid, axis=1))
    representative_words[cluster_id] = valid_words[cluster_indices[closest_word_index]]

# Mostrar los embeddings y los clusteres
plt.figure(figsize=(12, 8))
colors = plt.cm.rainbow(np.linspace(0, 1, num_clusters))

for i, word in enumerate(valid_words):
    cluster_id = kmeans.labels_[i]
    plt.scatter(word_vectors_2d[i, 0], word_vectors_2d[i, 1], color=colors[cluster_id])

# Mostrar palabras representativas
for cluster_id, representative_word in representative_words.items():
    representative_index = valid_words.index(representative_word)
    plt.scatter(word_vectors_2d[representative_index, 0], word_vectors_2d[representative_index, 1], 
                color=colors[cluster_id], edgecolor='black', linewidth=2, s=100, label=f'Cluster {cluster_id}: {representative_word}')
    plt.text(word_vectors_2d[representative_index, 0] + 0.1, word_vectors_2d[representative_index, 1], representative_word, fontsize=12)

# Leyenda con etiquetas de colores
legend_elements = []
for cluster_id in range(num_clusters):
    legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', label=f'Cluster {cluster_id}', 
                                      markerfacecolor=colors[cluster_id], markersize=10))

plt.legend(handles=legend_elements, loc='upper right', fontsize='large', bbox_to_anchor=(1.2, 1))
plt.title('Clustering de Palabras usando Word2Vec y K-Means')
plt.tight_layout()
plt.show()

OPCIÓN 1. Mega-etiquetas

In [ ]:
clusters_df = pd.read_excel('clusters_10.xlsx')
info_post_df = pd.read_excel('info_post.xlsx')
caracteristicas_df = pd.read_excel('caracteristicas.xlsx')

# Definir las nuevas columnas
new_columns = ['transporte', 'ropa', 'comida', 'mascota', 'varios', 'instrumentos musicales', 'animales', 'objetos', 'alimentos', 'hospital']
for col in new_columns:
    info_post_df[col] = 0

# Crear un diccionario para mapear las clases a sus clusters
class_to_cluster = dict(zip(clusters_df['Word'], clusters_df['Cluster']))

# Mapeo de cluster_id a nueva columna
cluster_to_column = {
    0: 'transporte',
    1: 'ropa',
    2: 'comida',
    3: 'perro/gato',
    4: 'varios',
    5: 'instrumentos musicales',
    6: 'animales',
    7: 'objetos',
    8: 'alimentos',
    9: 'hospital'
}

# Actualizar las nuevas columnas en info_post_df
for _, row in caracteristicas_df.iterrows():
    archivo = row['filename']
    clase = row['clase']
    if clase in class_to_cluster:
        cluster_id = class_to_cluster[clase]
        column_name = cluster_to_column.get(cluster_id, None)
        if column_name:
            info_post_df.loc[info_post_df['archivo'] == archivo, column_name] = 1

# Guardar el archivo actualizado
info_post_df.to_excel('info_post_prueba_etiquetas.xlsx', index=False)
print('Archivo info_post_prueba_etiquetas.xlsx generado exitosamente.')

OPCION 2. ETIQUETAS ANALIZADAS

In [ ]:
directorio = 'C:\\TFG\\imagenes_sin_texto'
im_sin_texto = [os.path.splitext(f)[0] for f in os.listdir(directorio) if f.endswith('.jpg')]

print(len(im_sin_texto))

df = pd.read_excel('C:\\TFG\\info_post.xlsx')
df_filtrado = df[df['archivo'].isin(im_sin_texto)]
df_filtrado.to_excel('C:\\TFG\\info_post_sin_texto.xlsx', index = False, engine = 'openpyxl')

In [ ]:
# Crear un diccionario para mapear las etiquetas a sus clases
etiquetas = {'escenario': 'stage',
             'microfono': 'microphone',
             'instrumento_percusion': ['gong', 'marimba', 'drum', 'drumstick', 'maraca'],
             'instrumento_cuerda': ['electric_guitar', 'acoustic_guitar', 'banjo', 'violin', 'cello', 'grand_piano', 'harp', 'organ'],
             'instrumento_viento': ['panpipe', 'sax', 'flute', 'basson', 'oboe', 'accordion', 'harmonica', 'cornet', 'trombone', 'ocarina'], 
             'mascota': ['toy_poodle', 'vizsla', 'miniature_poodle', 'Chihuahua', 'standard_poodle', 'Weimaraner', 'Sussex_spaniel', 'Great_Dane', 'Labrador_retriever', 
                         'German_shepherd', 'Yorkshire_terrier', 'miniature_pinscher', 'schipperke', 'American_Staffordshire_terrier', 'golden_retriever', 'cocker_spaniel', 
                         'Dandie_Dinmont', 'giant_schnauzer', 'llama', 'whippet', 'Afghan_hound', 'pug', 'Scottish_deerhound', 'Brittany_spaniel', 'Doberman', 'Pekinese', 
                         'bloodhound', 'Great_Pyrenees', 'basenji', 'basset', 'beagle', 'Irish_wolfhound', 'Pomeranian', 'Samoyed', 'Siamese_cat', 'kelpie', 'miniature_schnauzer',
                         'Rhodesian_ridgeback', 'Siberian_husky', 'tabby', 'dalmatian', 'malamute', 'Sealyham_terrier', 'bull_mastiff', 'Rottweiler', 'Tibetan_mastiff', 'boxer',
                         'silky_terrier', ' keeshond', 'Norfolk_terrier', 'Tibetan_terrier', 'Bernese_mountain_dog', 'papillon', 'Border_collie', 'collie', 'Shetland_sheepdog']
            }


In [ ]:
info_post_df = pd.read_excel('info_post_sin_texto.xlsx')
caracteristicas_df = pd.read_excel('caracteristicas.xlsx')

# Definir las nuevas columnas
new_columns = ['escenario', 'microfono', 'instrumento_percusion', 'instrumento_cuerda', 'instrumento_viento', 'mascota']
for col in new_columns:
    info_post_df[col] = 0

# Actualizar las nuevas columnas en info_post_df
for _, row in caracteristicas_df.iterrows():
    archivo = row['filename']
    print(archivo)
    clase = row['clase']
    print(clase)
    for etiqueta, palabras in etiquetas.items():
        if clase in palabras:
            info_post_df.loc[info_post_df['archivo'] == archivo, etiqueta] = 1
            print(etiqueta)

# Guardar el archivo actualizado
info_post_df.to_excel('info_post_sin_texto_etiquetas.xlsx', index=False)

print('Archivo info_post_etiquetas.xlsx generado exitosamente.')
